In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR100",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 60,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 128,
            'batch_size_memory': 128,
            'batch_size_validation': 256,
            'tau': 1,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.1,
            # 'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.000
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=CIFAR100/seed=1_epoch=60_lr=0.1_alpha=0.0_tau=1


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

# backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = cl.backbones.ResNet18Small(num_classes_per_head = 20, num_classes = 100, config=params).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
solver=<function LS_solver at 0x7f53279b4700>
[1] Eval metrics for task 1 >> {'accuracy': 20.049999999999997, 'loss': 0.010216673254966736, 'std': 21.74045767687516}
solver=<function LS_solver at 0x7f53279b4700>
[2] Eval metrics for task 1 >> {'accuracy': 30.800000000000004, 'loss': 0.00884446358680725, 'std': 19.45404842185811}
solver=<function LS_solver at 0x7f53279b4700>
[3] Eval metrics for task 1 >> {'accuracy': 36.4, 'loss': 0.008244039833545685, 'std': 18.467268341582084}
solver=<function LS_solver at 0x7f53279b4700>
[4] Eval metrics for task 1 >> {'accuracy': 38.10000000000001, 'loss': 0.008028327763080597, 'std': 21.926924088891266}
solver=<function LS_solver at 0x7f53279b4700>
[5] Eval metrics for task 1 >> {'accuracy': 42.25, 'loss': 0.0076158402562141415, 'std': 16.51022410508107}
solver=<function LS_solver at 0x7f53279b4700>
[6] Eval metrics for task 1 >> {'accuracy': 43.60000000000001, 'loss': 0.00713107758760452

In [5]:
metric_manager_callback.meters['std'].get_std()

[10.297936686540659,
 16.975552273784782,
 17.33971036538833,
 20.480112304379585,
 20.254737717383556]

In [6]:
metric_manager_callback.meters['accuracy'].get_data()

array([[71.55,  0.  ,  0.  ,  0.  ,  0.  ],
       [53.9 , 76.75,  0.  ,  0.  ,  0.  ],
       [45.55, 59.1 , 75.25,  0.  ,  0.  ],
       [36.7 , 45.35, 58.6 , 72.55,  0.  ],
       [36.5 , 48.05, 54.2 , 55.15, 80.3 ]])

In [7]:
metric_manager_callback.meters['accuracy'].compute_overall()

[71.55, 65.32499999999999, 59.96666666666667, 53.3, 54.839999999999996]

In [8]:
np.mean(metric_manager_callback.meters['std'].get_std())

17.069609869495384